# New York Housing Data Set

Analysis by John Bonfardeci
jbonfardeci@definitivelogic.com
2018-12-11

In [ ]:
require(lattice) # plotting lib required for car package
library(car) # includes vif(), scatterplotMatrix()
library(ggplot2) # plotting
#library(MASS)
library(dplyr) # common utilities for filtering
library(glmnet) # LASSO
library(caret) # includes methods for LASSO selection, trainControl CV k-fold

#install.packages('dummies')
library(dummies) # for converting categorical variables into indicator/dummy variables

## Question

**What variables in the data set predict the Full Market Value of a house?**

### Sample: Housing Data Set

In [ ]:
# Import housing data set into dataframe instance.
df = read.csv("housing.csv")
head(df)

### Explore: Univariate Analysis
* Look for skewness in histograms
    * A large empty space to the left is "skewed right."
    * A large empty space to the right is "skewed left."
    * If skewed, will log-transforming the variable make the distribution more normal (bell-shapped)?
* Look for outliers in the box plots.

In [ ]:
# Display quantiles of variables in the housing data set.
summary(df)

In [ ]:
boro <- df$Boro

# counts
ggplot(data.frame(boro), aes(x=boro)) +
  geom_bar() + labs(title="Count of Boro")

In [ ]:
neighborhood <- df$Neighborhood
ggplot(data.frame(neighborhood), aes(x=neighborhood)) +
  geom_bar() + labs(title="Count of Neighborhood")

In [ ]:
building_class <- df$BuildingClassification
ggplot(data.frame(building_class), aes(x=building_class)) +
  geom_bar() + labs(title="Count of Building Classification")

In [ ]:
par(mfrow=c(2, 2))

hist(df$TotalUnits, main = "Total Units", xlab = "Total Units")

boxplot(df$TotalUnits, main = "Total Units",
    xlab = "Units",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logTotalUnits <- log(df$TotalUnits)

hist(df$logTotalUnits, main = "Log(Total Units)", xlab = "Log(Total Units)")

boxplot(df$logTotalUnits, main = "Log(Total Units)",
    xlab = "Log(Units)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

In [ ]:
par(mfrow=c(2, 2))

hist(df$GrossSqFt, main = "Gross Sq. Ft.")

boxplot(df$GrossSqFt, main = "Gross Sq. Ft.",
    xlab = "Sq. Ft.",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logGrossSqFt <- log(df$GrossSqFt)

hist(df$logGrossSqFt, main = "Log(Gross Sq. Ft.)")

boxplot(df$logGrossSqFt, main = "Log(Gross Sq. Ft.)",
    xlab = "Log(Sq. Ft.)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

In [ ]:
par(mfrow=c(2, 2))

hist(df$EstimatedGrossIncome, main = "Estimated Gross Income")

boxplot(df$EstimatedGrossIncome, main = "Estimated Gross Income",
    xlab = "Estimated Gross Income",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logEstimatedGrossIncome <- log(df$EstimatedGrossIncome)

hist(df$logEstimatedGrossIncome, main = "Log(Estimated Gross Income)")

boxplot(df$logEstimatedGrossIncome, main = "Log(Estimated Gross Income)",
    xlab = "Log(Estimated Gross Income)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

In [ ]:
par(mfrow=c(2, 2))

hist(df$GrossIncomePerSqFt, main = "Gross Income/Sq. Ft.")

boxplot(df$GrossIncomePerSqFt, main = "Gross Income/Sq. Ft.",
    xlab = "Gross Income/Sq. Ft.",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logGrossIncomePerSqFt <- log(df$GrossIncomePerSqFt)

hist(df$logGrossIncomePerSqFt, main = "Log(Gross Income/Sq. Ft.)")

boxplot(df$logGrossIncomePerSqFt, main = "Log(Gross Income/Sq. Ft.)",
    xlab = "Log(Gross Income/Sq. Ft.)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

In [ ]:
par(mfrow=c(2, 2))

hist(df$EstimatedExpense, main = "Estimated Expense")

boxplot(df$EstimatedExpense, main = "Estimated Expense",
    xlab = "Estimated Expense",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logEstimatedExpense <- log(df$EstimatedExpense)

hist(df$logEstimatedExpense, main = "Log(Estimated Expense)")

boxplot(df$logEstimatedExpense, main = "Log(Estimated Expense)",
    xlab = "Log(Estimated Expense)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

In [ ]:
par(mfrow=c(2, 2))

hist(df$ExpensePerSqFt, main = "Expense/Sq. Ft.")

boxplot(df$ExpensePerSqFt, main = "Expense/Sq. Ft.",
    xlab = "Expense/Sq. Ft.",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logExpensePerSqFt <- log(df$ExpensePerSqFt)

hist(df$logExpensePerSqFt, main = "Log(Expense/Sq. Ft.)")

boxplot(df$logExpensePerSqFt, main = "Log(Expense/Sq. Ft.)",
    xlab = "Log(Expense/Sq. Ft.)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

In [ ]:
par(mfrow=c(2, 2))

hist(df$NetOperatingIncome, main = "Net Operating Income")

boxplot(df$NetOperatingIncome, main = "Net Operating Income",
    xlab = "Net Operating Income",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logNetOperatingIncome <- log(df$NetOperatingIncome)

hist(df$logNetOperatingIncome, main = "Log(Net Operating Income)")

boxplot(df$logNetOperatingIncome, main = "Log(Net Operating Income)",
    xlab = "Log(Net Operating Income)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

In [ ]:
par(mfrow=c(2, 2))

hist(df$FullMarketValue, main = "Full Market Value")

boxplot(df$FullMarketValue, main = "Full Market Value",
    xlab = "Full Market Value",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

df$logFullMarketValue <- log(df$FullMarketValue)

hist(df$logFullMarketValue, main = "Log(Full Market Value)")

boxplot(df$logFullMarketValue, main = "Log(Full Market Value)",
    xlab = "Log(Full Market Value)",
    col = "orange",
    border = "brown",
    horizontal = TRUE,
    notch = TRUE)

## Multivariate Analysis

In [ ]:
# Drop non-numerical variables.
numerical <- subset(df, select = -c(ID, Neighborhood, BuildingClassification, Boro, YearBuilt, 
                                    logTotalUnits, logGrossSqFt, logEstimatedGrossIncome, logGrossIncomePerSqFt,
                                    logEstimatedExpense, logExpensePerSqFt, logNetOperatingIncome, logFullMarketValue,
                                    logGrossIncomePerSqFt, GrossIncomePerSqFt, logExpensePerSqFt, ExpensePerSqFt, MarketValuePerSqFt)) 

# Display a scatter plot matrix to identify linear relationships between variables.
scatterplotMatrix(numerical, main="Scatter Plot Matrix") 
#spread=FALSE, smoother.args=list(lty=2), 

In [ ]:
# Show Correlation Matrix
cor(numerical)
# Positive numbers indicate a linear relationship between two variables.
# Numbers ~0 indicate little to no relationship.
# Numbers < 0 indicate inverse correlation.

### Split the dataset into 70% training and 30% validation sets. 

In [ ]:

# 70% of sample
sample_size <- floor(0.7 * nrow(df))

# set seed to make partitions reproducible.
set.seed(42)
train_ind <- sample( seq_len(nrow(df)), size = sample_size)

# Get training set
train_data <- df[train_ind, ]

# Get validation set
validation <- df[-train_ind, ]


### Model

In [ ]:
# Fit linear regression model with lm()
fit <- lm(FullMarketValue ~ EstimatedGrossIncome, data = train_data)
plot(train_data$EstimatedGrossIncome, train_data$FullMarketValue, xlab = "Estimated Gross Income", ylab = "Full Market Value", 
     main = "Full Market Value Model")
abline(fit, col = "red", lty=1, lwd=2)

### Assess

In [ ]:
summary(fit)
bic = BIC(fit)
paste("BIC", ":", BIC(fit))

### Asjusted R-squared is .97 meaning the model accounts for 97% of the variance in EstimatedGrossIncome.

In [ ]:
# Show plot of standardized residuals by the fitted values. If non-constant variance, 
# increasing variance from left to right, the model is not valid.
par(mfrow=c(2,2))
plot(fit, pch = 18)

### Is the model valid?
* **Linearity** - (Top Left)
    * If Y is linearly related to X, there should be no relationship between the residuals and th efitted values. 
    * The scatter plot should show be random noise.
    * Since there's increasing variance from left to right, the linearity assumption is violated.

* **Normality** - (Top Right)
    * If Y is normally distrubuted, then the residual values should be normally distributed with a mean of 0.
    * Q-Q plot is a probability plot of the std. residuals against the values that would be expected under normality.
    * The points do not fall on this line, so the normality assumption is violated.

* **Homoscedasticity** (Bottom Left)
    * A plot of the std. residuals against the fitted values should show constant variance (random noise) about a horizontal line.
    * Assumption of Homoscedasticity is violated.
    
* **Outliers** - Std. Residuals against Cook's Distance
    * Observations with Std. Residual < -3 or > 3 have a *high leverage* value. They're outliers.
    * Observations that have a Cook's Distance value > 1 are overly influential.
    
**The model is INVALID!**

## How to we fix an invalid model?
We know from the histograms that both the response variable and the predictor are not normally distributed - they're both skewed. We know that log-transforming can make a variable normally distributed, so we try log-transforming the response variable and the predictor.

### Modify

In [ ]:
# Log-transform the X
par(mfrow=c(2,1))
hist(df$logEstimatedGrossIncome, main = "Log(Estimated Gross Income)")
boxplot(df$logEstimatedGrossIncome, main = "Log(Estimated Gross Income)", horizontal = TRUE)
# The histogram displayed shows a more normal distribution than the non-log-transformed version.

In [ ]:
# Log-transform the Y
par(mfrow=c(2,1))
hist(log(df$logFullMarketValue), main = "Log(Full Market Value)")
boxplot(log(df$logFullMarketValue), main = "Log(Full Market Value)", horizontal = TRUE)
# The histogram displayed shows a more normal distribution than the non-log-transformed version.

### Model: Re-fit the model with the log-transformed variables.

In [ ]:
# Fit a new model with the log-transformed versions.
log_fit <- lm(logFullMarketValue ~ logEstimatedGrossIncome, data = train_data)
plot(train_data$logEstimatedGrossIncome, train_data$logFullMarketValue, xlab = "Log(Estimated Gross Income)", ylab = "Log(Full Market Value)")
abline(log_fit, col = "red", lty=1, lwd=2)

### Assess

Notice the elliptical symmetry about the regression line.

In [ ]:
summary(log_fit)
bic2 <- BIC(log_fit)
paste("BIC 1", ":", bic)
paste("BIC 2", ":", bic2)

### Summary of Fit
* Adjusted R-squared is 0.98 - `logEstimatedGrossIncome` accounts for 98% of the variance in `logFullMarketValue`.
* BIC (Bayesian Information Criterion) decreased from 60847 to -866.7 Lower is better.
* The p-value for the y-intercept is significant (p-value < 0.05)
* Predictor `logEstimatedGrossIncome` is significant (p-value < 0.05)

### Do we have constant variance?

In [ ]:
# Test for constant variance
ncvTest(log_fit)

The p-value from `ncvTest()` is **not significant** (> 0.05), suggesting we've met the assumuption of constant variance.
With the exception of outliers, it would appear we've **met the assumption of constant variance**.

In [ ]:
# Show plot of residuals by the fitted values. Do we have constant variance?
par(mfrow=c(2,2))
plot(log_fit, pch = 18)

In [ ]:
summary(log_fit)

### Formula
Based on the `summary()` output above, we have the Y-intercept (Beta 0) and coefficient of Beta 1. 

Our formula is:

`Y-hat = 0.700055 + 1.056609(X1), where Y-hat = Log(FullMarketValue), X1 = Log(EstimatedGrossIncome)`

### Interpretaion of the Model

When log-transforming a dependent variable and/or predictor, our interpretation of the model will change.
Normally an in/decrease in one unit of X corresponds to an in/decrease in one unit of Y.
When dealing with log-transforms for both Y and X, the interpretation is:

**A 1% increase in Log(EstimatedGrossIncome) is predicted to increase Log(FullMarketValue) by 1.057%.**

## Outliers

In [ ]:
outlierTest(log_fit)

## Leverage Points
* A *leverage point* is a point whose x-value is distant from the other x-values.
* Data points which exercise considerable influence on the fitted model.
* Not all leverage points are bad leverage points.
* All bad leverage points are outliers but not all outliers are bad leverage points.
* We'll determine if a leverage point is bad using Cook's Distance

### Standardized Residual (aka, Studentized Residual)
* a residual divided by it's standard deviation.
* As a rule, a studentized residual < -3 or > 3 is a bad leverage point.
* In very large data sets, a more appropriate cuttoff is < -4 or > 4, or even < -5 or > 5

In [ ]:
# Get studentized residuals < -3 or > 3
stud_resid <- rstandard(log_fit)

# Show rows where Std. Resid. < -3 or > 3
train_data$StdResid <- stud_resid
r <- filter(train_data, (train_data$StdResid < -3 | train_data$StdResid > 3))
r

### Cook's Distance
* R. Dennis Cook. University of Minnesota. 1977. (https://en.wikipedia.org/wiki/R._Dennis_Cook)
* Cook's Distance is a composite measure of outlyingness and leverage.
* Rule of thumb (They're more like guidelines.):
    * Cook's Distance > 1 - investigate. Adding a dummy variable to these cases can change regression estimates.
    * Cases with Cook's Distance > 0.5 should also be investigated.
    * Also look for gaps om the values of Cook's distance and not just whether values exceed the suggested cuttoff. 

In [ ]:
# Get Cook's Distance points where > 1
getCooksD <- function(fit){
    stud_resid <- rstandard(fit)
    cooks_dist <- cooks.distance(fit)
    influential_points <- cooks_dist[cooks_dist >= 1] # Filter on points > 1 with a list comprehension.
    influential_points # Display influential points
    if( length(influential_points) == 0){
        print("There are no points with a Cook's D > 1.")
    }

    plot(x = cooks_dist, y = stud_resid, main="Std. Residuals by Cook's D", xlab="Cook's D", ylab = "Studentized Residuals")
}

getCooksD(log_fit)

Though there are observations with studentized residuals less than -3 and greater than 3, we do not need to worry that they're bad leverage points since none of the Cook's Distance values exceed 1. However those with a Cook's D > 0.5 should be investigated. Results of `outlierTest()` in this case should not be cause for concern.

## How do we handle bad leverage points?
Leverage points need to be investigated but not removed from a model. Doing so can cause us to lose important observations.

To negate the influence of a a bad leverage point and improve the model's predictive accuracy, we add what's called a "dummy variable" or "indicator variable." This variable has a value of "1" if it's a bad leverage point, and a "0" otherwise. Dummy variables will cause the beta estimates in the equation to change. This means if there's a bad leverage point in the training set, it won't change the way the model predicts new observations.

In [ ]:
# Artificially set a bad leverage point for this example.
train_data$logFullMarketValue[train_data$ID == 2569 ] <- 25

log_fit2 <- lm(logFullMarketValue ~ logEstimatedGrossIncome, data = train_data)
plot(train_data$logEstimatedGrossIncome, train_data$logFullMarketValue, xlab = "Log(Estimated Gross Income)", ylab = "Log(Full Market Value)")
abline(log_fit2, col = "red", lty=1, lwd=2)

summary(log_fit2)

In [ ]:
# Now we have a bad leverage point.
getCooksD(log_fit2)

In [ ]:
# We add a dummy variable = 1 if Cook's D >= 1.
train_data$CooksD <- cooks.distance(log_fit2)

# Create a Dummy colunm with default value 0.
train_data$Overpriced <- rep(0, nrow(train_data))

# Set the dummy == 1 on the bad leverage point.
train_data$Overpriced[train_data$ID == 2569 ] <- 1

head(train_data)

In [ ]:
# After adding the dummy variable, we see the regression line is no longer affected.
log_fit3 <- lm(logFullMarketValue ~ logEstimatedGrossIncome + Overpriced, data = train_data)
train_data$Yhat <- predict(log_fit3, train_data)

plot(train_data$Yhat, train_data$logFullMarketValue, xlab = "Predicted", ylab = "Actual")
abline(lm(logFullMarketValue ~ Yhat, data = train_data), col = "red", lty=1, lwd=2)

In [ ]:
# The coefficients after adding the dummy variable are not much different than the previous fit.
# Take note of the small differences in the Intercepts and estimated for `logEstimatedGrossIncome`.
print("Fit Without Bad Leverage Point")
summary(log_fit2)

print("Fit With Bad Leverage Point")
summary(log_fit3)

## Predict the Validation Dataset
Our formula is:
`y-hat <- 0.689401 + (1.057403 * x1)`

In [ ]:
predictMarketValue <- function(x1){
  return (0.689401 + (1.057403 * x1))
}

# Create a new column in validation called `Yhat` to store predictions from our formula.
validation$Yhat <- rep(0, nrow(validation))

# Predict Full Market Value each observation.
for(i in 1:nrow(validation)){
  egi <- validation$logEstimatedGrossIncome[i]
  validation$Yhat[i] <- predictMarketValue(egi)
}

# Show top 10 results.
head(validation)

#predictions <- predict(log_fit3, validation)

#validation$Yhat <- predictions

In [ ]:
plot(validation$Yhat, validation$logFullMarketValue, xlab = "Predicted", ylab = "Actual")
abline(lm(logFullMarketValue ~ Yhat, data = validation), col = "red", lty=1, lwd=2)

# Export the validation set.
#write.csv(validation, "validation.csv")

## Multiple Linear Regression

### Convert Categorical Variables to Indicator Columns

### Add New Variable for Age
Imputation of Missing (NA) Values
* There are NA values in the YearBuilt column.
* Determine if percentage of YearBuilt == NA is acceptable to replace with the mean.

In [ ]:
nrows <- nrow(df)

# Get mean of YearBuilt excluding NA values.
missing_year_count <- nrow( filter(df, is.na(YearBuilt)) )
perc_missing <- missing_year_count / nrows * 100

print(paste("Missing", round(perc_missing, 2), "% of YearBuilt values."))

In [ ]:
# Impute missing values for YearBuilt
mean_year <- floor( mean(df$YearBuilt[!is.na(df$YearBuilt)]) )
current_year <- 2018

for(i in 1:nrow(df)){
    yr <- df$YearBuilt[i]
    age <- 0
    
    if(is.na(yr)){
        age <- current_year - mean_year
    } else {
        age <- current_year - yr
    }
    
    df$Age[i] <- age
}

head(df)

### Convert Categorical Variables to Indicator Columns

In [ ]:
df_subset <- subset(df, select = -c(TotalUnits, YearBuilt, GrossSqFt, EstimatedGrossIncome, 
                                    GrossIncomePerSqFt, EstimatedExpense, ExpensePerSqFt, 
                                    NetOperatingIncome, FullMarketValue, MarketValuePerSqFt, 
                                    logGrossIncomePerSqFt, 
                                    logExpensePerSqFt, Neighborhood)) 

head(df_subset)

In [ ]:
df2 <- dummy.data.frame(df_subset, names=c("Boro", "BuildingClassification"), sep="_")
names(df2) <- make.names(names(df2)) # remove special characters from column names
head(df2)

In [ ]:
# Get training set
train_data <- df2[train_ind, ]
#write.csv(train_data, "train.csv")

# Get validation set
validation2 <- df2[-train_ind, ]
#write.csv(validation2, "validation.csv")

train_data <- subset(train_data, select = -c(ID, logNetOperatingIncome))

### Feature Selection with LASSO
* Least Absolute Shrinkage & Selection Operator
* Reg. Analys that performs feature selection
* Makes the model simpler and more interpretable
* Shrinks regression coefficients, with some shrunk to zero. Zero coefficients are removed (feature selection).

In [ ]:
# LASSO
lasso <- train(logFullMarketValue ~ .,
               data = train_data,
               tuneGrid = expand.grid(alpha=1, lambda = seq(0.0001, 0.2, length = 5)),
               method = "glmnet")

### Variable Importance
Shown in order of importance. Those equal to zero are not important.

In [ ]:
# Variable Importance plot
plot(varImp(lasso, scale=F))

In [ ]:
# 0 or less is not important.
varImp(lasso)

In [ ]:
# Re-run linrar regression with the variables LASSO selected.
# Use variables that LASSO selected
lm2 <- train(logFullMarketValue ~ logEstimatedGrossIncome  + 
    logEstimatedExpense +
    Boro_Staten.Island +
    logGrossSqFt + 
    Boro_Bronx +
    BuildingClassification_RR.CONDOMINIUM + 
    Boro_Queens,
    data = train_data, 
    method = "lm")

In [ ]:
summary(lm2$finalModel)
# All variables should have a p-value < 0.05 and are significant.

### Variance Inflation Factors (VIFs)
* Measures multicollinearity - variables that to highly related.
* Variables with a VIF > 10 will cause a model to be unstable. 
* Small changes in variable values will cause wild shifts in the model.
* Goal: remove or combine variables with VIF > 10

In [ ]:
vif(lm2$finalModel)

In [ ]:
# Remove variables with high multicollinearity.
# Re-fit linear regression model.
lm3 <- train(logFullMarketValue ~ logEstimatedGrossIncome +
    Boro_Staten.Island +
    Boro_Bronx +
    BuildingClassification_RR.CONDOMINIUM + 
    Boro_Queens,
    data = train_data, 
    method = "lm")

vif(lm3$finalModel)
summary(lm3$finalModel)

In [ ]:
# Do we have a valid model?
par(mfrow=c(2,2))
plot(lm3$finalModel)

In [ ]:
ncvTest(lm3$finalModel)
# p-value is not significant. Cosntant variance assumption seems to have been met.

### Do we have bad leverage points?

In [ ]:
getCooksD(lm3$finalModel)

While there are clear outliers, none are overly influential; none have a Cook's Distance > 1 and removing them or adding a dummy variable for these observations will not significant change the estimated coefficient.

## Final Linear Regression Model Fit with Significant Variables

In [ ]:
predicted <- predict(lm2, validation2)
validation2$Yhat <- predicted

plot(validation2$Yhat, validation2$logFullMarketValue, xlab = "Predicted", ylab = "Actual")
abline(lm(logFullMarketValue ~ Yhat, data = validation2), col = "red", lty=1, lwd=2)